# Contexts

## Computational contexts

### Functor

#### `<$>`

In [169]:
from Contexts.Functor import ListFunctor

a=ListFunctor([1,2,3,4])
a.fmap(float).fmap(str)

['1.0', '2.0', '3.0', '4.0']

In [170]:
from Contexts.Functor import TupleFunctor

a=TupleFunctor(tuple([1,2,3,4]))
a.fmap(float).fmap(str)

('1.0', '2.0', '3.0', '4.0')

In [171]:
print(a)

(1, 2, 3, 4)


In [172]:
a=ListFunctor([])
a.fmap(float)

[]

In [173]:
a=TupleFunctor(())
a.fmap(float)

()

#### `<$`: map replace-by

In [174]:
def const(a,b):
    return a

In [175]:
from functools import partial
def replaceby(a):
    return partial(const,a)

list(map(replaceby([1,2]),[1,2,3,4]))

[[1, 2], [1, 2], [1, 2], [1, 2]]

In [176]:
a=ListFunctor([1,2,3,4])
a.fmap(replaceby([1,2,3]))

[[1, 2, 3], [1, 2, 3], [1, 2, 3], [1, 2, 3]]

### Applicative

In [177]:
from Contexts.Applicative import ListApplicative

a=ListApplicative([1,2,3,4])
a.ap([lambda x: -x]).ap([float,abs])

[-1.0, -2.0, -3.0, -4.0, 1, 2, 3, 4]

In [178]:
a.then([1,2])

[1, 2, 1, 2, 1, 2, 1, 2]

In [179]:
a.pure(3).ap([float,abs])

[3.0, 3]

In [180]:
from Contexts.Applicative import pure, liftA2, ap

pure([3])

[3]

In [181]:
from operator import add

list(liftA2(add,[1,2],[3,4]))

[4, 5, 5, 6]

In [182]:
from Basic.Monoid import *

a.ap(pure([partial(add,1)]))

[2, 3, 4, 5]

In [183]:
a.ap([partial(add,1)])

[2, 3, 4, 5]

In [184]:
a.ap([partial(y,x) for x,y in zip([1,2],[add]*2)])

[2, 3, 4, 5, 3, 4, 5, 6]

In [185]:
a=ListApplicative([a])

In [186]:
a.ap([lambda x:x[:2],lambda x:x[2:]])

[[1, 2], [3, 4]]

In [187]:
type(a+a)

list

In [188]:
b = mappend(a,a)
type(b)

Contexts.Applicative.ListApplicative

#### Empty applicative

In [189]:
c=mempty(b)

In [190]:
type(c)

Contexts.Applicative.ListApplicative

The `pure` method still work for an empty applicative:

In [191]:
c.pure(3)

[3]

In [192]:
c.then([1,2])

[]

Beautiful! So in the empty applicative context, replacing other contents still resulting in empty.

### Monad

In [193]:
from Contexts.Monad import ListMonad

a=ListMonad([1,2,3,4])
a.ap([lambda x: -x]).ap([float,abs])

[-1.0, -2.0, -3.0, -4.0, 1, 2, 3, 4]

In [194]:
a.fmap(lambda x: [x-1,x,x+1])

[[0, 1, 2], [1, 2, 3], [2, 3, 4], [3, 4, 5]]

In [195]:
a.bind(lambda x: [x-1,x,x+1])

[0, 1, 2, 1, 2, 3, 2, 3, 4, 3, 4, 5]

In [196]:
a.bind(lambda x: [x-1,x,x+1]).bind(lambda x: [x] if (x%2==0) else [])

[0, 2, 2, 2, 4, 4]

In [197]:
type(mappend(a,a))

Contexts.Monad.ListMonad

## Interator contexts

### Foldable

In [198]:
from Contexts.Foldable import ListFoldable, foldr, foldr_r, scanl, scanl_r, scanr, scanr_r

a=ListFoldable([1,2,3,4])
a.foldl((lambda acc,x: acc+x),0)

10

In [199]:
a.foldr((lambda x,acc: x+acc),0)

10

In [200]:
foldr((lambda x,acc:x+acc),(1,2,3,4),0)

10

In [201]:
next(foldr_r((lambda x,acc:x+acc),(1,2,3,4),0))

10

In [202]:
list(scanl((lambda acc,x: acc+x),0,(1,2,3,4)))

[1, 3, 6, 10]

In [203]:
def leftEdge(n,acc,x): return (acc[1][-n:],x)
def rightEdge(n,x,acc): return x+(acc[1][:n],)

In [204]:
a=[[1,2,3,4],[5,6,7,8]]
r2=partial(rightEdge,2)
l2=partial(leftEdge,2)
foo=scanl(l2,([],[-1,0]),a)
print(foo)

[([-1, 0], [1, 2, 3, 4]), ([3, 4], [5, 6, 7, 8])]


In [205]:
scanr(r2,foo,([],[9,10],[]))[::-1]

[([-1, 0], [1, 2, 3, 4], [5, 6]), ([3, 4], [5, 6, 7, 8], [9, 10])]

```{note}
The structure of the initial value determines the structure of the element in the new container. This is because `scanl` (or `scanr`) take an input function with type `[[B,A],A]`, that is mapping element with type `B` (initial value) and element in the container with type `A` to output value with type `B`. In the above example, the shape of the new element is `(list,list)` for the `l2` function, while for `r2`, the shape is `(list,list,list)`.
```

In [206]:

foo=list(scanl_r(l2,([],[-1,0]),a))
print(foo)

[([-1, 0], [1, 2, 3, 4]), ([3, 4], [5, 6, 7, 8])]


In [207]:
print(a)

[[1, 2, 3, 4], [5, 6, 7, 8]]


In [208]:
list(scanr_r(r2,foo,([],[9,10],[])))[::-1]

[([-1, 0], [1, 2, 3, 4], [5, 6]), ([3, 4], [5, 6, 7, 8], [9, 10])]